<h1><center>Physique numérique (PHY-3500)</center></h1>
<h1><center>TP3 - Équations différentielles</center></h1>

## Nom des membres de l'équipe:

### Nom, Prénom :  NI 
### Légaré, Ophélie : 111 182 608
### Parrot, Anaïs : 111 184 980
### Turgeon, David : 111 182 265

### Importation des modules :

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy as atp
from astropy.time import Time
%pip install jplephem
from jplephem import Ephemeris
%pip install de421
import de421

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Introduction : 

# Mécanique céleste - comètes

## Question a.
Il est possible de séparer une équation différentielle d'ordre supérieur en plusieurs équations différentielles de premier ordre. La deuxième loi de Newton qui décrit le mouvement bidimensionnel d'une comète autour du soleil est exprimée par deux équations différentielles de deuxième ordre. On sépare ces deux équations pour les ramener à quatre équations différentielles du premier ordre.

La démarche détaillée est présentée pour le mouvement dans la direction $x$ seulement puisque la démarche pour le mouvement en $y$ est la même. 

L'équation différentielle de départ est
$$\frac{d^{2}x}{dt^{2}} = -GM \frac{x}{(x^{2}+y^{2})^{\frac{3}{2}}} = f(x, y).$$
On commence par faire des changements de variable pour chaque degré en $x$.
$$\delta = \frac{dx}{dt}= x'$$
$$\delta' = \frac{d \delta}{dt} = \frac{d^{2}x}{dt^{2}} = x''$$
On substitue ces changements de variable dans l'équation du mouvement et on obtient deux équations différentielles de premier ordre.
$$\delta = \frac{dx}{dt}$$
$$\frac{d \delta}{dt} = -GM \frac{x}{(x^{2}+y^{2})^{\frac{3}{2}}}$$

Dans le cas du mouvement en $y$, on obtient une réponse équivalente avec un changement de variable différent.
$$\gamma = \frac{dy}{dt}$$
$$\frac{d \gamma}{dt} = -GM \frac{y}{(x^{2}+y^{2})^{\frac{3}{2}}}$$

## Question b.

## Question c.

# Mécanique céleste - la Terre

## Question d.

## Question e.

## Question f.

## Question g.

# Mécanique céleste - Mars

## Question h.

In [15]:
# éphémérides
eph = Ephemeris(de421)
# dates
lancement=Time("2020-07-30")
atterissage=Time("2021-02-18")
# un nombre de jours juliens est attendu par la routine, d’où le .jd
# position en km, vitesse en km par jour
position, velocity = eph.position_and_velocity('mars',lancement.jd)
eph.position('mars',atterissage.jd)

array([[-1.94538523e+06],
       [ 2.14280417e+08],
       [ 9.83090643e+07]])